<a href="https://colab.research.google.com/github/tazar09/heroes_3/blob/main/h3_heroes%5B25.oct2023%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
df1 = pd.read_html('https://heroes.thelazy.net/index.php/List_of_heroes_(HotA)')

In [404]:
df1[0].head()

,Name,Class,Unnamed: 2,Specialty,Unnamed: 4,Skill 1,Unnamed: 6,Skill 2,Unnamed: 8,Spell
0,Anabel,Captain,NaN,Pirates,NaN,Basic Offense,NaN,Basic Archery,NaN,NaN
1,Cassiopeia,Captain,NaN,Nymphs,NaN,Basic Offense,NaN,Basic Tactics,NaN,NaN
2,Corkes,Captain,NaN,Offense,NaN,Basic Offense,NaN,Basic Pathfinding,NaN,NaN
3,Derek,Captain,NaN,Crew Mates,NaN,Basic Offense,NaN,Basic Leadership,NaN,NaN
4,Elmore,Captain,NaN,Navigation,NaN,Advanced Navigation,NaN,NaN,NaN,NaN


In [554]:
df = df1[0].copy()
df = df.drop(axis = 1, labels = ['Unnamed: 2','Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8'])
df.columns = ['name', 'class', 'specialty', 'skill_1', 'skill_2', 'spell']

In [555]:
skill1 = df['skill_1'].str.split(expand = True, n = 1)
skill2 = df['skill_2'].str.split(expand = True, n = 1)
df = pd.concat([df, skill1, skill2], axis = 1)
df.columns = ['name', 'class', 'specialty', 'skill_1', 'skill_2', 'spell', 'skill_1_level', 'skill_1_name', 'skill_2_level', 'skill_2_name']

In [556]:
df = df.drop(axis = 1, labels = ['skill_1', 'skill_2'])

In [557]:
df = df[['name', 'class', 'specialty', 'skill_1_name', 'skill_1_level', 'skill_2_name','skill_2_level', 'spell']]
df['spell'] = df['spell'].fillna('')
df['skill_2_name'] = df['skill_2_name'].fillna('')
df['skill_2_level'] = df['skill_2_level'].fillna('')
df['count'] = 1
df['spell_tf'] = df['spell'].apply(lambda x: 0 if x == '' else 1)
df.head()

,name,class,specialty,skill_1_name,skill_1_level,skill_2_name,skill_2_level,spell,count,spell_tf
0,Anabel,Captain,Pirates,Offense,Basic,Archery,Basic,,1,0
1,Cassiopeia,Captain,Nymphs,Offense,Basic,Tactics,Basic,,1,0
2,Corkes,Captain,Offense,Offense,Basic,Pathfinding,Basic,,1,0
3,Derek,Captain,Crew Mates,Offense,Basic,Leadership,Basic,,1,0
4,Elmore,Captain,Navigation,Navigation,Advanced,,,,1,0


In [558]:
towns = pd.read_html('http://heroes.thelazy.net/index.php/List_of_all_town_names')
towns = list(towns[1]['Town'].unique())

In [559]:
towns = ['Castle', 'Rampart', 'Tower', 'Inferno', 'Necropolis','Dungeon', 'Stronghold' ,'Fortress','Conflux', 'Cove']

In [560]:
classes = list(df['class'].unique())
classes.remove('Captain')
classes.remove('Navigator')
classes.insert(18, 'Captain')
classes.insert(19, 'Navigator')
classes[8] = 'Death Knight'

In [561]:
towns2 = []
temp_list = []
for i in towns:
  temp_list = [i, i]
  towns2.append(temp_list)
towns2x = [element for sublist in towns2 for element in sublist]
len(classes), len(towns2x)

(20, 20)

In [562]:
class_towns = pd.DataFrame({'class': classes, 'town': towns2x})
df = df.merge(class_towns, how='left', on='class')

In [563]:
df.columns = ['name', 'class', 'specialty', 'skill_1', 'level_1',
       'skill_2', 'level_2', 'spell', 'count', 'spell_tf', 'town']

In [566]:
df['class_2'] = df['skill_1'].apply(lambda x: 'magic' if x == 'Wisdom' else 'might')
df.iloc[81, 11] = 'might'  #Rashka
df.iloc[101, 11] = 'magic' #Aislinn

In [568]:
df = df[['name', 'class', 'class_2', 'specialty', 'level_1', 'skill_1',
       'level_2', 'skill_2', 'spell', 'count', 'spell_tf', 'town']]

In [570]:
df.head(5)

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,spell,count,spell_tf,town
0,Anabel,Captain,might,Pirates,Basic,Offense,Basic,Archery,,1,0,Cove
1,Cassiopeia,Captain,might,Nymphs,Basic,Offense,Basic,Tactics,,1,0,Cove
2,Corkes,Captain,might,Offense,Basic,Offense,Basic,Pathfinding,,1,0,Cove
3,Derek,Captain,might,Crew Mates,Basic,Offense,Basic,Leadership,,1,0,Cove
4,Elmore,Captain,might,Navigation,Advanced,Navigation,,,,1,0,Cove


In [597]:
creatures = pd.read_csv('/content/df_h3.csv')
creatures = creatures[['Name', 'Upgraded']]

In [605]:
creatures['specialty'] = creatures['Name'].apply(lambda x: x + 's')
creatures['specialty'] = creatures['specialty'].apply(lambda x: 'convert' if x.endswith('mans') else x)
creatures.head()

,Name,Upgraded,specialty
0,Pikeman,0,convert
1,Halberdier,1,Halberdiers
2,Archer,0,Archers
3,Marksman,1,convert
4,Griffin,0,Griffins


In [606]:
creatures[creatures['specialty'] == 'convert']

,Name,Upgraded,specialty
0,Pikeman,0,convert
3,Marksman,1,convert
6,Swordsman,0,convert
100,Lizardman,0,convert
129,Seaman,1,convert


In [590]:
df.merge(creatures, how = 'left', on = 'specialty')

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,spell,count,spell_tf,town,upgraded,names
0,Anabel,Captain,might,Pirates,Basic,Offense,Basic,Archery,,1,0,Cove,NaN,NaN
1,Cassiopeia,Captain,might,Nymphs,Basic,Offense,Basic,Tactics,,1,0,Cove,NaN,NaN
2,Corkes,Captain,might,Offense,Basic,Offense,Basic,Pathfinding,,1,0,Cove,NaN,NaN
3,Derek,Captain,might,Crew Mates,Basic,Offense,Basic,Leadership,,1,0,Cove,NaN,NaN
4,Elmore,Captain,might,Navigation,Advanced,Navigation,,,,1,0,Cove,NaN,NaN
5,Illor,Captain,might,Stormbirds,Basic,Armorer,Basic,Tactics,,1,0,Cove,NaN,NaN
6,Jeremy,Captain,might,Cannon,Basic,Offense,Basic,Artillery,,1,0,Cove,NaN,NaN
7,Leena,Captain,might,Gold,Basic,Pathfinding,Basic,Estates,,1,0,Cove,NaN,NaN
8,Miriam,Captain,might,Scouting,Basic,Logistics,Basic,Scouting,,1,0,Cove,NaN,NaN
9,Bidley,Captain,might,Sea Dogs,Advanced,Offense,,,,1,0,Cove,NaN,NaN
